#创建spider，以及爬取对象
#爬取图片的主题为“女孩”
import scrapy
from ..items import PictureItem
from protego import Protego
from selenium import webdriver
from scrapy.selector import Selector
from selenium.webdriver.chrome.options import Options
import time

chorme_options = Options()
chorme_options.add_argument("--headless")
chorme_options.add_argument("--disable-gpu")

class MySpider(scrapy.Spider):
    #用于区别Spider
    name = "MySpider"
    #允许访问的域
    allowed_domains = []
    #爬取的地址
    start_urls = ["https://gracg.com/search/index_new?key=%E5%B0%91%E5%A5%B3"]

    # 实例化一个浏览器对象
    def __init__(self):
        self.browser = webdriver.Chrome(chrome_options=chorme_options)
        super().__init__()

    def start_requests(self):
        #读取前4页的图片数据信息
        for urlnum in range(1,6):
            url = "https://gracg.com/search/index_new?key=%E5%B0%91%E5%A5%B3&ot=0&type=1&page="+str(urlnum)

            headers={
                        'Connection': 'keep-alive',
                         'Cache-Control': 'max-age=0',
                         'DNT': '1',
                        'Upgrade-Insecure-Requests': '1',
                        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'

                    }
            response = scrapy.Request(url,headers=headers, callback=self.parse_index)
            yield response

    # 整个爬虫结束后关闭浏览器
    def close(self, spider):
        self.browser.quit()

    #爬取简介页面方法
    def parse_index(self, response):
        #实例一个容器保存爬取的信息
        filename = "picture.html"
        with open(filename, 'wb') as f:
            f.write(response.body)
        f.close()
        for box in response.xpath('//div[@class="container"]/div[@class="row"]/div/div[1]/div[1]/a[2]'):
            url = 'http://gracg.com' + box.xpath('.//@href').extract()[0]
            print("url",url)
            # 获取详情页面的url并用parse_detail方法解析
            response=scrapy.Request(url,callback=self.parse_detail)
            time.sleep(0.1)
            yield response

    # 爬取详情页面方法
    def parse_detail(self, response):
        with open("item.html", 'wb') as f:
            f.write(response.body)
        f.close()
        div_res = response.xpath('//div[@class=" col-12 col-lg-9   pl-0  pr-0 "]')
        name = div_res.xpath('.//div[@class="font-weight-bold font-24  "]/text()').extract()
        print("name",name)
        count = div_res.xpath('.//div[@class="d-flex flex-row font-12   pb-2"]')
        seen = count.xpath('./div[1]/span/text()').extract()
        thumb_up = count.xpath('./div[2]/span/text()').extract()
        comment = count.xpath('./div[3]/span/text()').extract()
        tags = div_res.xpath('.//a[@class=" border-50 p-2 font-12 rounded d-block mr-1 mb-1 "]/text()')[1:].extract()
        # 存入Item中并发送给pipeline
        item = PictureItem()
        item["name"]=name
        item["seen"] = seen
        item["thumb_up"] = thumb_up
        item["comment"] = comment
        item["tags"] = tags
        yield item



#在scrapy文件夹下的iterms文件里写出要爬取的数据
#其他的配置文件没有拷贝进来

import scrapy

class PictureItem(scrapy.Item):

    name = scrapy.Field()#图片名
    seen = scrapy.Field()#观看数
    thumb_up=scrapy.Field()#点赞数
    comment = scrapy.Field()#评论数
    tags = scrapy.Field()#标签种类

#运行main.py文件
from scrapy.cmdline import execute
import os
import sys

sys.path.append(os.path.dirname(os.path.abspath(__file__)))
execute(['scrapy','crawl','MySpider'])

#爬下来的数据运行结果
{'comment': ['0'],
 'name': [' 【Q版人物】动作练习：开演唱会啦'],
 'seen': ['34'],
 'tags': ['板绘', '手绘', '少女', '人物', 'Q版'],
 'thumb_up': ['0']}
name [' 【Q版人物】动作练习：开演唱会啦']
2021-01-25 15:29:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1658081> (referer: None)
name [' 古装少女插画']
2021-01-25 15:29:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1658081>
{'comment': ['0'],
 'name': [' 古装少女插画'],
 'seen': ['144'],
 'tags': ['古风', '少女', '发饰', '竹叶'],
 'thumb_up': ['0']}
2021-01-25 15:29:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1658074> (referer: None)
name [' 【人物头像】那个女人']
2021-01-25 15:29:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1658074>
{'comment': ['0'],
 'name': [' 【人物头像】那个女人'],
 'seen': ['114'],
 'tags': ['插画', '头像', '少女心', '每日一画', '打卡', '儿童插画'],
 'thumb_up': ['2']}
2021-01-25 15:29:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1658054> (referer: None)
name [' 临摹']
2021-01-25 15:29:35 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1658054>
{'comment': ['0'],
 'name': [' 临摹'],
 'seen': ['154'],
 'tags': ['procreate', '打卡', '少女', '人物', '练习', '临摹'],
 'thumb_up': ['3']}
2021-01-25 15:29:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1658040> (referer: None)
name [' 原来你是这样的葡萄']
2021-01-25 15:29:37 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1658040>
{'comment': ['0'],
 'name': [' 原来你是这样的葡萄'],
 'seen': ['170'],
 'tags': ['插画', '少女', '练习'],
 'thumb_up': ['3']}
2021-01-25 15:29:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1658028> (referer: None)
name [' 漫画版簪花仕女']
2021-01-25 15:29:38 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1658028>
{'comment': ['0'],
 'name': [' 漫画版簪花仕女'],
 'seen': ['67'],
 'tags': ['绘本', '古风', '少女', '插画', '可爱', 'Q版', '中国风'],
 'thumb_up': ['3']}
2021-01-25 15:29:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1658021> (referer: None)
2021-01-25 15:29:40 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1658021>
{'comment': ['0'],
 'name': [' 路过冬季街头咖啡小铺'],
 'seen': ['213'],
 'tags': ['板绘', '冬天', '原创', '街景', '少女插画', '打卡', '水彩', '绘本', '手绘', 'ps', '每日一画'],
 'thumb_up': ['5']}
name [' 路过冬季街头咖啡小铺']
2021-01-25 15:29:42 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1658016> (referer: None)
name [' 少女插画']
2021-01-25 15:29:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1658016>
{'comment': ['0'],
 'name': [' 少女插画'],
 'seen': ['198'],
 'tags': ['插画', '清新'],
 'thumb_up': ['4']}
2021-01-25 15:29:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657976> (referer: None)
name [' 古风人物厚涂插画']
2021-01-25 15:29:45 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657976>
{'comment': ['0'],
 'name': [' 古风人物厚涂插画'],
 'seen': ['142'],
 'tags': ['板绘', '古风', '少女', '厚涂'],
 'thumb_up': ['5']}
2021-01-25 15:29:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657978> (referer: None)
2021-01-25 15:29:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657978>
{'comment': ['0'],
 'name': [' procreate插画｜少女下午茶'],
 'seen': ['147'],
 'tags': ['清新', '绿植', '少女', 'procreate', '插画'],
 'thumb_up': ['5']}
name [' procreate插画｜少女下午茶']
2021-01-25 15:29:48 [scrapy.extensions.logstats] INFO: Crawled 16 pages (at 16 pages/min), scraped 10 items (at 10 items/min)
2021-01-25 15:29:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657969> (referer: None)
2021-01-25 15:29:49 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657969>
{'comment': ['0'],
 'name': [' 幻书启世录'],
 'seen': ['172'],
 'tags': ['幻书启世录', '少女'],
 'thumb_up': ['3']}
name [' 幻书启世录']
2021-01-25 15:29:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657968> (referer: None)
name [' 十二生肖拟人系列']
2021-01-25 15:29:51 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657968>
{'comment': ['0'],
 'name': [' 十二生肖拟人系列'],
 'seen': ['183'],
 'tags': ['国风', '人物', '插画', '传统', '生肖'],
 'thumb_up': ['4']}
2021-01-25 15:29:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657962> (referer: None)
name [' 叮～～']
2021-01-25 15:29:52 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657962>
{'comment': ['0'],
 'name': [' 叮～～'],
 'seen': ['132'],
 'tags': ['每日一画', '打卡', '少女', '可爱', 'Q版', '人设', '二次元'],
 'thumb_up': ['4']}
2021-01-25 15:29:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657827> (referer: None)
name [' 关于贵族少女是否流泪的这件事']
2021-01-25 15:29:54 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657827>
{'comment': ['0'],
 'name': [' 关于贵族少女是否流泪的这件事'],
 'seen': ['268'],
 'tags': [],
 'thumb_up': ['3']}
2021-01-25 15:29:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657567> (referer: None)
2021-01-25 15:29:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657567>
{'comment': ['1'],
 'name': [' 大明风华'],
 'seen': ['442'],
 'tags': ['影视', '人物', '壁纸', '绘画', '插画', 'oc', '水彩'],
 'thumb_up': ['6']}
name [' 大明风华']
2021-01-25 15:29:58 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657374> (referer: None)
2021-01-25 15:29:58 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657374>
{'comment': ['0'],
 'name': [' Biubiu~'],
 'seen': ['243'],
 'tags': ['头像', '少女', '厚涂'],
 'thumb_up': ['5']}
name [' Biubiu~']
2021-01-25 15:29:59 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656687> from <GET http://gracg.com/works/view/1656687>
2021-01-25 15:30:00 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656690> from <GET http://gracg.com/works/view/1656690>
2021-01-25 15:30:03 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656700> from <GET http://gracg.com/works/view/1656700>
2021-01-25 15:30:05 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656713> from <GET http://gracg.com/works/view/1656713>
2021-01-25 15:30:07 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656749> from <GET http://gracg.com/works/view/1656749>
2021-01-25 15:30:09 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656762> from <GET http://gracg.com/works/view/1656762>
2021-01-25 15:30:11 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656767> from <GET http://gracg.com/works/view/1656767>
2021-01-25 15:30:13 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656798> from <GET http://gracg.com/works/view/1656798>
2021-01-25 15:30:14 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656868> from <GET http://gracg.com/works/view/1656868>
2021-01-25 15:30:16 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656879> from <GET http://gracg.com/works/view/1656879>
2021-01-25 15:30:18 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656900> from <GET http://gracg.com/works/view/1656900>
2021-01-25 15:30:20 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656905> from <GET http://gracg.com/works/view/1656905>
2021-01-25 15:30:22 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657071> from <GET http://gracg.com/works/view/1657071>
2021-01-25 15:30:23 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656958> from <GET http://gracg.com/works/view/1656958>
2021-01-25 15:30:25 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657120> from <GET http://gracg.com/works/view/1657120>
2021-01-25 15:30:27 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656961> from <GET http://gracg.com/works/view/1656961>
2021-01-25 15:30:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656687> (referer: None)
name [' 【人物头像】少女']
2021-01-25 15:30:29 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656687>
{'comment': ['0'],
 'name': [' 【人物头像】少女'],
 'seen': ['305'],
 'tags': ['原创', '打卡', '每日一画', '少女', '插画', '板绘', '头像'],
 'thumb_up': ['4']}
2021-01-25 15:30:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656690> (referer: None)
2021-01-25 15:30:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656690>
{'comment': ['0'],
 'name': [' 桃花妖'],
 'seen': ['336'],
 'tags': ['插画', '少女'],
 'thumb_up': ['3']}
name [' 桃花妖']
2021-01-25 15:30:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656700> (referer: None)
2021-01-25 15:30:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656700>
{'comment': ['0'],
 'name': [' 少女穿搭'],
 'seen': ['403'],
 'tags': ['少女', '可爱', '人物', '清新'],
 'thumb_up': ['3']}
name [' 少女穿搭']
2021-01-25 15:30:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656713> (referer: None)
name [' 版权保护']
2021-01-25 15:30:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656713>
{'comment': ['0'],
 'name': [' 版权保护'],
 'seen': ['280'],
 'tags': ['少女', '插画', '女孩', '涂鸦'],
 'thumb_up': ['4']}
2021-01-25 15:30:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656749> (referer: None)
name [' 坚持摸鱼（1/100）']
2021-01-25 15:30:35 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656749>
{'comment': ['0'],
 'name': [' 坚持摸鱼（1/100）'],
 'seen': ['587'],
 'tags': ['打卡', 'procreate', '插画', '原创', '少女', '板绘', '头像'],
 'thumb_up': ['10']}
2021-01-25 15:30:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656762> (referer: None)
2021-01-25 15:30:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656762>
{'comment': ['0'],
 'name': [' 尝试学习国画创作ing……'],
 'seen': ['368'],
 'tags': ['procreate', '水彩', '原创', '少女', '水墨'],
 'thumb_up': ['5']}
name [' 尝试学习国画创作ing……']
2021-01-25 15:30:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656767> (referer: None)
name [' 无题']
2021-01-25 15:30:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656767>
{'comment': ['0'],
 'name': [' 无题'],
 'seen': ['321'],
 'tags': ['美少女', '绘画'],
 'thumb_up': ['5']}
2021-01-25 15:30:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656798> (referer: None)
name [' 【原创】向日葵']
2021-01-25 15:30:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656798>
{'comment': ['0'],
 'name': [' 【原创】向日葵'],
 'seen': ['183'],
 'tags': ['绘本', '打卡', '少女', '可爱', '儿童插画', 'Q版'],
 'thumb_up': ['4']}
2021-01-25 15:30:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656868> (referer: None)
2021-01-25 15:30:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656868>
{'comment': ['0'],
 'name': [' 祝你新年快乐呀'],
 'seen': ['477'],
 'tags': ['procreate', '板绘', '少女', '头像', '手绘', '绘画', '人物'],
 'thumb_up': ['5']}
name [' 祝你新年快乐呀']
2021-01-25 15:30:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656879> (referer: None)
name [' 精致少女']
2021-01-25 15:30:45 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656879>
{'comment': ['0'],
 'name': [' 精致少女'],
 'seen': ['1074'],
 'tags': [],
 'thumb_up': ['15']}
2021-01-25 15:30:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656900> (referer: None)
2021-01-25 15:30:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656900>
{'comment': ['0'],
 'name': [' 我们都在海里'],
 'seen': ['462'],
 'tags': ['插画', '原创', '板绘', '少女', '绘画', '日系', '二次元'],
 'thumb_up': ['5']}
name [' 我们都在海里']
2021-01-25 15:30:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656905> (referer: None)
name [' 森女系插画']
2021-01-25 15:30:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656905>
{'comment': ['0'],
 'name': [' 森女系插画'],
 'seen': ['320'],
 'tags': ['手绘', '少女'],
 'thumb_up': ['4']}
2021-01-25 15:30:48 [scrapy.extensions.logstats] INFO: Crawled 34 pages (at 18 pages/min), scraped 28 items (at 18 items/min)
2021-01-25 15:30:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657071> (referer: None)
name [' 星空下的少女']
2021-01-25 15:30:49 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657071>
{'comment': ['0'],
 'name': [' 星空下的少女'],
 'seen': ['400'],
 'tags': [],
 'thumb_up': ['5']}
2021-01-25 15:30:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656958> (referer: None)
2021-01-25 15:30:51 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656958>
{'comment': ['0'],
 'name': [' ►'],
 'seen': ['320'],
 'tags': ['少女', 'Procreate', '打卡', '女孩', '人物', '绘画'],
 'thumb_up': ['4']}
name [' ►']
2021-01-25 15:30:53 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657120> (referer: None)
name [' 一个单眼皮朋友']
2021-01-25 15:30:53 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657120>
{'comment': ['0'],
 'name': [' 一个单眼皮朋友'],
 'seen': ['334'],
 'tags': ['闺蜜头像', '少女插画', '插画'],
 'thumb_up': ['4']}
2021-01-25 15:30:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656961> (referer: None)
name [' 旋']
2021-01-25 15:30:55 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656961>
{'comment': ['10'],
 'name': [' 旋'],
 'seen': ['4184'],
 'tags': ['版绘', '少女', '双人', '小清新', '游乐园', '旋转木马'],
 'thumb_up': ['103']}
2021-01-25 15:30:57 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657124> from <GET http://gracg.com/works/view/1657124>
2021-01-25 15:30:59 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656967> from <GET http://gracg.com/works/view/1656967>
2021-01-25 15:31:01 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657129> from <GET http://gracg.com/works/view/1657129>
2021-01-25 15:31:02 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656975> from <GET http://gracg.com/works/view/1656975>
2021-01-25 15:31:04 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657154> from <GET http://gracg.com/works/view/1657154>
2021-01-25 15:31:06 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1656992> from <GET http://gracg.com/works/view/1656992>
2021-01-25 15:31:08 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657157> from <GET http://gracg.com/works/view/1657157>
2021-01-25 15:31:10 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657000> from <GET http://gracg.com/works/view/1657000>
2021-01-25 15:31:12 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657170> from <GET http://gracg.com/works/view/1657170>
2021-01-25 15:31:13 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657008> from <GET http://gracg.com/works/view/1657008>
2021-01-25 15:31:15 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657193> from <GET http://gracg.com/works/view/1657193>
2021-01-25 15:31:16 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657009> from <GET http://gracg.com/works/view/1657009>
2021-01-25 15:31:18 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657196> from <GET http://gracg.com/works/view/1657196>
2021-01-25 15:31:21 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657014> from <GET http://gracg.com/works/view/1657014>
2021-01-25 15:31:22 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657200> from <GET http://gracg.com/works/view/1657200>
2021-01-25 15:31:23 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657016> from <GET http://gracg.com/works/view/1657016>
2021-01-25 15:31:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657124> (referer: None)
name [' 不良少女']
2021-01-25 15:31:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657124>
{'comment': ['0'],
 'name': [' 不良少女'],
 'seen': ['232'],
 'tags': [],
 'thumb_up': ['5']}
2021-01-25 15:31:26 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656967> (referer: None)
2021-01-25 15:31:26 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656967>
{'comment': ['0'],
 'name': [' 《等风》'],
 'seen': ['1964'],
 'tags': ['原创', '绘画', '少女', '场景', '唯美', '清新', '治愈'],
 'thumb_up': ['40']}
name [' 《等风》']
2021-01-25 15:31:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657129> (referer: None)
name [' 随便画画']
2021-01-25 15:31:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657129>
{'comment': ['0'],
 'name': [' 随便画画'],
 'seen': ['222'],
 'tags': ['头像', '少女', '女孩', '可爱', '厚涂'],
 'thumb_up': ['4']}
2021-01-25 15:31:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656975> (referer: None)
2021-01-25 15:31:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656975>
{'comment': ['4'],
 'name': [' 樱桃'],
 'seen': ['1575'],
 'tags': ['少女', '樱桃', '粉色', '每日一画', '礼物'],
 'thumb_up': ['33']}
name [' 樱桃']
2021-01-25 15:31:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657154> (referer: None)
name [' 情侣色块插画']
2021-01-25 15:31:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657154>
{'comment': ['0'],
 'name': [' 情侣色块插画'],
 'seen': ['1106'],
 'tags': ['青春', '少女', '场景'],
 'thumb_up': ['15']}
2021-01-25 15:31:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1656992> (referer: None)
name [' 棒棒糖少女']
2021-01-25 15:31:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1656992>
{'comment': ['0'],
 'name': [' 棒棒糖少女'],
 'seen': ['314'],
 'tags': [],
 'thumb_up': ['5']}
2021-01-25 15:31:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657157> (referer: None)
name [' 戏水']
2021-01-25 15:31:35 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657157>
{'comment': ['0'],
 'name': [' 戏水'],
 'seen': ['289'],
 'tags': ['少女', '治愈', '插画'],
 'thumb_up': ['4']}
2021-01-25 15:31:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657000> (referer: None)
name [' 自创人物头像']
2021-01-25 15:31:37 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657000>
{'comment': ['0'],
 'name': [' 自创人物头像'],
 'seen': ['264'],
 'tags': ['少女', '绘画', '人物', 'Procreate', '女孩'],
 'thumb_up': ['4']}
2021-01-25 15:31:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657170> (referer: None)
name [' 数字创作3']
2021-01-25 15:31:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657170>
{'comment': ['0'],
 'name': [' 数字创作3'],
 'seen': ['423'],
 'tags': ['原创', 'procreate', '商业插画', '场景', '唯美', '人像', '少女'],
 'thumb_up': ['5']}
2021-01-25 15:31:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657008> (referer: None)
2021-01-25 15:31:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657008>
{'comment': ['0'],
 'name': [' 深海蓝鱼'],
 'seen': ['143'],
 'tags': ['插画', '手绘', '封面', '原创', 'Procreate'],
 'thumb_up': ['5']}
name [' 深海蓝鱼']
2021-01-25 15:31:42 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657193> (referer: None)
2021-01-25 15:31:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657193>
{'comment': ['0'],
 'name': [' 愉悦的步伐'],
 'seen': ['228'],
 'tags': ['procreate', '插画', '原创', '少女', '涂鸦'],
 'thumb_up': ['3']}
name [' 愉悦的步伐']
2021-01-25 15:31:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657009> (referer: None)
name [' 林中少女']
2021-01-25 15:31:45 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657009>
{'comment': ['0'],
 'name': [' 林中少女'],
 'seen': ['186'],
 'tags': ['原创', '插画', '板绘', '绘画', '少女', '植物'],
 'thumb_up': ['4']}
2021-01-25 15:31:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657196> (referer: None)
name [' 鸽子']
2021-01-25 15:31:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657196>
{'comment': ['0'],
 'name': [' 鸽子'],
 'seen': ['180'],
 'tags': ['二次元', '插画', '板绘', '求约稿', '厚涂', '美少女'],
 'thumb_up': ['3']}
2021-01-25 15:31:48 [scrapy.extensions.logstats] INFO: Crawled 51 pages (at 17 pages/min), scraped 45 items (at 17 items/min)
2021-01-25 15:31:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657014> (referer: None)
2021-01-25 15:31:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657014>
{'comment': ['0'],
 'name': [' 海蝴蝶'],
 'seen': ['200'],
 'tags': ['人鱼', '封面', '手绘', 'procreate'],
 'thumb_up': ['4']}
name [' 海蝴蝶']
2021-01-25 15:31:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657200> (referer: None)
2021-01-25 15:31:50 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657200>
{'comment': ['0'],
 'name': [' 静静看着你'],
 'seen': ['435'],
 'tags': ['可爱', '女孩', '少女', '板绘'],
 'thumb_up': ['4']}
name [' 静静看着你']
2021-01-25 15:31:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657016> (referer: None)
name [' 白日梦']
2021-01-25 15:31:52 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657016>
{'comment': ['0'],
 'name': [' 白日梦'],
 'seen': ['255'],
 'tags': ['封面', '白日梦', 'procreate', '手绘', '绘图', '插画'],
 'thumb_up': ['5']}
2021-01-25 15:31:53 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657213> from <GET http://gracg.com/works/view/1657213>
2021-01-25 15:31:56 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657034> from <GET http://gracg.com/works/view/1657034>
2021-01-25 15:31:58 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657218> from <GET http://gracg.com/works/view/1657218>
2021-01-25 15:32:00 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657044> from <GET http://gracg.com/works/view/1657044>
2021-01-25 15:32:02 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657356> from <GET http://gracg.com/works/view/1657356>
2021-01-25 15:32:03 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657223> from <GET http://gracg.com/works/view/1657223>
2021-01-25 15:32:05 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657371> from <GET http://gracg.com/works/view/1657371>
2021-01-25 15:32:07 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657233> from <GET http://gracg.com/works/view/1657233>
2021-01-25 15:32:08 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657242> from <GET http://gracg.com/works/view/1657242>
2021-01-25 15:32:10 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657400> from <GET http://gracg.com/works/view/1657400>
2021-01-25 15:32:12 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657252> from <GET http://gracg.com/works/view/1657252>
2021-01-25 15:32:14 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657419> from <GET http://gracg.com/works/view/1657419>
2021-01-25 15:32:16 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657255> from <GET http://gracg.com/works/view/1657255>
2021-01-25 15:32:17 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657428> from <GET http://gracg.com/works/view/1657428>
2021-01-25 15:32:20 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657256> from <GET http://gracg.com/works/view/1657256>
2021-01-25 15:32:21 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657452> from <GET http://gracg.com/works/view/1657452>
2021-01-25 15:32:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657213> (referer: None)
2021-01-25 15:32:23 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657213>
{'comment': ['0'],
 'name': [' 手绘头像'],
 'seen': ['273'],
 'tags': ['原创', '少女', '古风', '头像', '可爱'],
 'thumb_up': ['4']}
name [' 手绘头像']
2021-01-25 15:32:25 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657034> (referer: None)
name [' Twice 林娜琏&amp;SANA']
2021-01-25 15:32:25 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657034>
{'comment': ['0'],
 'name': [' Twice 林娜琏&amp;SANA'],
 'seen': ['237'],
 'tags': ['偶像', '少女'],
 'thumb_up': ['4']}
2021-01-25 15:32:27 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657218> (referer: None)
name [' 心有猛虎，细嗅蔷薇']
2021-01-25 15:32:27 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657218>
{'comment': ['0'],
 'name': [' 心有猛虎，细嗅蔷薇'],
 'seen': ['496'],
 'tags': ['手绘', '少女', '原创', '人物', '女孩', '清新', '植物'],
 'thumb_up': ['5']}
2021-01-25 15:32:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657044> (referer: None)
name [' 水彩']
2021-01-25 15:32:29 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657044>
{'comment': ['0'],
 'name': [' 水彩'],
 'seen': ['416'],
 'tags': ['水彩', '女孩', '可爱', '商业插画', '绘本', '设计', '治愈'],
 'thumb_up': ['5']}
2021-01-25 15:32:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657356> (referer: None)
2021-01-25 15:32:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657356>
{'comment': ['0'],
 'name': [' 无题'],
 'seen': ['237'],
 'tags': ['少女', '手绘'],
 'thumb_up': ['4']}
name [' 无题']
2021-01-25 15:32:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657223> (referer: None)
2021-01-25 15:32:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657223>
{'comment': ['0'],
 'name': [' 《十二生肖-羊》'],
 'seen': ['455'],
 'tags': ['打卡', '插画', '原创', '板绘', '绘画', '少女', '人物'],
 'thumb_up': ['5']}
name [' 《十二生肖-羊》']
2021-01-25 15:32:35 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657371> (referer: None)
2021-01-25 15:32:35 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657371>
{'comment': ['0'],
 'name': [' 茉莉花茶'],
 'seen': ['842'],
 'tags': ['少女', '水彩', '插画'],
 'thumb_up': ['6']}
name [' 茉莉花茶']
2021-01-25 15:32:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657233> (referer: None)
2021-01-25 15:32:37 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657233>
{'comment': ['0'],
 'name': [' 今天就画一张人像手机壁纸吧'],
 'seen': ['284'],
 'tags': ['每日一画', '原创', '少女', '人像', '二次元', '女孩', '帅气'],
 'thumb_up': ['6']}
name [' 今天就画一张人像手机壁纸吧']
2021-01-25 15:32:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657242> (referer: None)
name [' 女孩子']
2021-01-25 15:32:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657242>
{'comment': ['0'],
 'name': [' 女孩子'],
 'seen': ['1567'],
 'tags': ['插画'],
 'thumb_up': ['23']}
2021-01-25 15:32:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657400> (referer: None)
2021-01-25 15:32:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657400>
{'comment': ['0'],
 'name': [' '],
 'seen': ['194'],
 'tags': ['procreate', '原创', '插画', '少女', '扁平风'],
 'thumb_up': ['4']}
name [' ']
2021-01-25 15:32:42 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657252> (referer: None)
2021-01-25 15:32:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657252>
{'comment': ['0'],
 'name': [' 画照片'],
 'seen': ['344'],
 'tags': ['手绘', '绘画', '头像', '少女'],
 'thumb_up': ['6']}
name [' 画照片']
2021-01-25 15:32:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657419> (referer: None)
name [' 没有收尾']
2021-01-25 15:32:45 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657419>
{'comment': ['0'],
 'name': [' 没有收尾'],
 'seen': ['302'],
 'tags': ['头像', '少女', '女孩', '人像', 'ipad', '可爱'],
 'thumb_up': ['4']}
2021-01-25 15:32:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657255> (referer: None)
name [' 日常']
2021-01-25 15:32:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657255>
{'comment': ['0'],
 'name': [' 日常'],
 'seen': ['612'],
 'tags': ['少女'],
 'thumb_up': ['15']}
2021-01-25 15:32:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657428> (referer: None)
2021-01-25 15:32:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657428>
{'comment': ['0'],
 'name': [' 猫猫女'],
 'seen': ['247'],
 'tags': ['原创', '插画', '少女', '人物', 'sai', '日系'],
 'thumb_up': ['4']}
name [' 猫猫女']
2021-01-25 15:32:48 [scrapy.extensions.logstats] INFO: Crawled 68 pages (at 17 pages/min), scraped 62 items (at 17 items/min)
2021-01-25 15:32:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657256> (referer: None)
2021-01-25 15:32:49 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657256>
{'comment': ['0'],
 'name': [' 临摹'],
 'seen': ['289'],
 'tags': [],
 'thumb_up': ['5']}
name [' 临摹']
2021-01-25 15:32:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657452> (referer: None)
name [' 抹茶温泉']
2021-01-25 15:32:50 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657452>
{'comment': ['0'],
 'name': [' 抹茶温泉'],
 'seen': ['343'],
 'tags': ['手绘', '水彩', '少女', '梦幻', '可爱'],
 'thumb_up': ['4']}
2021-01-25 15:32:52 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657257> from <GET http://gracg.com/works/view/1657257>
2021-01-25 15:32:55 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657455> from <GET http://gracg.com/works/view/1657455>
2021-01-25 15:32:56 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657274> from <GET http://gracg.com/works/view/1657274>
2021-01-25 15:32:58 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657456> from <GET http://gracg.com/works/view/1657456>
2021-01-25 15:33:00 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657275> from <GET http://gracg.com/works/view/1657275>
2021-01-25 15:33:02 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657481> from <GET http://gracg.com/works/view/1657481>
2021-01-25 15:33:04 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657286> from <GET http://gracg.com/works/view/1657286>
2021-01-25 15:33:06 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657526> from <GET http://gracg.com/works/view/1657526>
2021-01-25 15:33:08 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657318> from <GET http://gracg.com/works/view/1657318>
2021-01-25 15:33:10 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657528> from <GET http://gracg.com/works/view/1657528>
2021-01-25 15:33:11 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657348> from <GET http://gracg.com/works/view/1657348>
2021-01-25 15:33:13 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657542> from <GET http://gracg.com/works/view/1657542>
2021-01-25 15:33:15 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657674> from <GET http://gracg.com/works/view/1657674>
2021-01-25 15:33:17 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657591> from <GET http://gracg.com/works/view/1657591>
2021-01-25 15:33:19 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657678> from <GET http://gracg.com/works/view/1657678>
2021-01-25 15:33:22 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657594> from <GET http://gracg.com/works/view/1657594>
2021-01-25 15:33:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657257> (referer: None)
name [' 临摹']
2021-01-25 15:33:23 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657257>
{'comment': ['0'],
 'name': [' 临摹'],
 'seen': ['274'],
 'tags': [],
 'thumb_up': ['4']}
2021-01-25 15:33:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657455> (referer: None)
name [' 新生']
2021-01-25 15:33:25 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657455>
{'comment': ['0'],
 'name': [' 新生'],
 'seen': ['303'],
 'tags': ['插画', '原创', '少女', '人物', '女孩'],
 'thumb_up': ['4']}
2021-01-25 15:33:26 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657274> (referer: None)
2021-01-25 15:33:26 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657274>
{'comment': ['0'],
 'name': [' 白天的月亮'],
 'seen': ['340'],
 'tags': ['插画', '原创', '人物', '每日一画', '打卡', '少女', '绘画'],
 'thumb_up': ['5']}
name [' 白天的月亮']
2021-01-25 15:33:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657456> (referer: None)
name [' 少女']
2021-01-25 15:33:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657456>
{'comment': ['0'],
 'name': [' 少女'],
 'seen': ['289'],
 'tags': [],
 'thumb_up': ['5']}
2021-01-25 15:33:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657275> (referer: None)
name [' 【人物头像】少女']
2021-01-25 15:33:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657275>
{'comment': ['0'],
 'name': [' 【人物头像】少女'],
 'seen': ['345'],
 'tags': ['原创', '插画', '儿童插画', '少女', '每日一画', '头像', '板绘'],
 'thumb_up': ['5']}
2021-01-25 15:33:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657481> (referer: None)
2021-01-25 15:33:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657481>
{'comment': ['0'],
 'name': [' 水彩练习'],
 'seen': ['235'],
 'tags': ['水彩', '临摹', '少女'],
 'thumb_up': ['5']}
name [' 水彩练习']
2021-01-25 15:33:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657286> (referer: None)
2021-01-25 15:33:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657286>
{'comment': ['0'],
 'name': [' 水彩，女孩与蛇'],
 'seen': ['276'],
 'tags': ['水彩', '少女', '蛇', '手绘'],
 'thumb_up': ['4']}
name [' 水彩，女孩与蛇']
2021-01-25 15:33:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657526> (referer: None)
name [' 落书']
2021-01-25 15:33:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657526>
{'comment': ['0'],
 'name': [' 落书'],
 'seen': ['303'],
 'tags': ['少女', '插画', '板绘'],
 'thumb_up': ['5']}
2021-01-25 15:33:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657318> (referer: None)
2021-01-25 15:33:38 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657318>
{'comment': ['0'],
 'name': [' 向日葵和少女'],
 'seen': ['687'],
 'tags': ['清新', '扁平', '植物', '板绘'],
 'thumb_up': ['11']}
name [' 向日葵和少女']
2021-01-25 15:33:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657528> (referer: None)
2021-01-25 15:33:40 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657528>
{'comment': ['0'],
 'name': [' 水彩人物'],
 'seen': ['349'],
 'tags': ['水彩', '绘本', '少女', '插画', '清新', '治愈', '植物'],
 'thumb_up': ['4']}
name [' 水彩人物']
2021-01-25 15:33:42 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657348> (referer: None)
2021-01-25 15:33:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657348>
{'comment': ['0'],
 'name': [' 眼镜娘'],
 'seen': ['198'],
 'tags': ['插画', '美少女'],
 'thumb_up': ['5']}
name [' 眼镜娘']
2021-01-25 15:33:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657542> (referer: None)
2021-01-25 15:33:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657542>
{'comment': ['0'],
 'name': [' '],
 'seen': ['190'],
 'tags': ['procreate', '原创', '插画', '头像', '少女', '绘画'],
 'thumb_up': ['5']}
name [' ']
2021-01-25 15:33:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657674> (referer: None)
name [' 数字创作4']
2021-01-25 15:33:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657674>
{'comment': ['0'],
 'name': [' 数字创作4'],
 'seen': ['413'],
 'tags': ['原创', '插画', '少女', '商业插画', '场景'],
 'thumb_up': ['7']}
2021-01-25 15:33:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657591> (referer: None)
name [' 水彩插画']
2021-01-25 15:33:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657591>
{'comment': ['0'],
 'name': [' 水彩插画'],
 'seen': ['323'],
 'tags': ['每日一画', '少女', '清新', '治愈'],
 'thumb_up': ['5']}
2021-01-25 15:33:48 [scrapy.extensions.logstats] INFO: Crawled 84 pages (at 16 pages/min), scraped 78 items (at 16 items/min)
2021-01-25 15:33:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657678> (referer: None)
2021-01-25 15:33:49 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657678>
{'comment': ['0'],
 'name': [' 光影女孩'],
 'seen': ['360'],
 'tags': ['每日一画', '板绘', '少女', '头像'],
 'thumb_up': ['10']}
name [' 光影女孩']
2021-01-25 15:33:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657594> (referer: None)
2021-01-25 15:33:51 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657594>
{'comment': ['0'],
 'name': [' 周末少女'],
 'seen': ['187'],
 'tags': ['原创', '少女', '绘画', '女孩', '涂鸦', '绘本', '漫画'],
 'thumb_up': ['5']}
name [' 周末少女']
2021-01-25 15:33:53 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657681> from <GET http://gracg.com/works/view/1657681>
2021-01-25 15:33:54 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657618> from <GET http://gracg.com/works/view/1657618>
2021-01-25 15:33:56 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657687> from <GET http://gracg.com/works/view/1657687>
2021-01-25 15:33:58 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657621> from <GET http://gracg.com/works/view/1657621>
2021-01-25 15:33:59 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657689> from <GET http://gracg.com/works/view/1657689>
2021-01-25 15:34:02 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657626> from <GET http://gracg.com/works/view/1657626>
2021-01-25 15:34:03 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657690> from <GET http://gracg.com/works/view/1657690>
2021-01-25 15:34:05 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657630> from <GET http://gracg.com/works/view/1657630>
2021-01-25 15:34:07 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657722> from <GET http://gracg.com/works/view/1657722>
2021-01-25 15:34:08 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657632> from <GET http://gracg.com/works/view/1657632>
2021-01-25 15:34:10 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657745> from <GET http://gracg.com/works/view/1657745>
2021-01-25 15:34:12 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657642> from <GET http://gracg.com/works/view/1657642>
2021-01-25 15:34:14 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657784> from <GET http://gracg.com/works/view/1657784>
2021-01-25 15:34:15 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657656> from <GET http://gracg.com/works/view/1657656>
2021-01-25 15:34:17 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657788> from <GET http://gracg.com/works/view/1657788>
2021-01-25 15:34:19 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657657> from <GET http://gracg.com/works/view/1657657>
2021-01-25 15:34:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657681> (referer: None)
name [' 小圆']
2021-01-25 15:34:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657681>
{'comment': ['0'],
 'name': [' 小圆'],
 'seen': ['108'],
 'tags': ['小圆'],
 'thumb_up': ['4']}
2021-01-25 15:34:23 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657618> (referer: None)
name [' 黑历史重绘']
2021-01-25 15:34:23 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657618>
{'comment': ['2'],
 'name': [' 黑历史重绘'],
 'seen': ['2332'],
 'tags': ['procreate', '古风', '头像', '少女', '人物', '中国风', '女孩'],
 'thumb_up': ['33']}
2021-01-25 15:34:25 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657687> (referer: None)
name [' 人设-花']
2021-01-25 15:34:25 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657687>
{'comment': ['0'],
 'name': [' 人设-花'],
 'seen': ['391'],
 'tags': ['原创', '古风', '少女'],
 'thumb_up': ['6']}
2021-01-25 15:34:27 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657621> (referer: None)
2021-01-25 15:34:27 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657621>
{'comment': ['0'],
 'name': [' 【2020下半年印花图案设计作品合集】'],
 'seen': ['306'],
 'tags': ['印花', '服装印花', '碎花', '可爱', '少女'],
 'thumb_up': ['4']}
name [' 【2020下半年印花图案设计作品合集】']
2021-01-25 15:34:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657689> (referer: None)
2021-01-25 15:34:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657689>
{'comment': ['0'],
 'name': [' 一些图！'],
 'seen': ['103'],
 'tags': ['魔法少女小圆',
          '物语系列',
          '忍野忍',
          '午睡留声机',
          '明日之子',
          '头像',
          '约稿',
          '摸鱼',
          '巴麻美',
          '鹿目圆香'],
 'thumb_up': ['5']}
name [' 一些图！']
2021-01-25 15:34:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657626> (referer: None)
2021-01-25 15:34:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657626>
{'comment': ['0'],
 'name': [' 一个可爱小姐姐的约稿'],
 'seen': ['258'],
 'tags': ['插画', '原创', '少女', '头像', '女孩', '可爱'],
 'thumb_up': ['5']}
name [' 一个可爱小姐姐的约稿']
2021-01-25 15:34:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657690> (referer: None)
name [' 梦幻小女巫~']
2021-01-25 15:34:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657690>
{'comment': ['0'],
 'name': [' 梦幻小女巫~'],
 'seen': ['256'],
 'tags': ['原创', '少女', '可爱', '星空'],
 'thumb_up': ['6']}
2021-01-25 15:34:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657630> (referer: None)
name [' 尤克里里小姐姐']
2021-01-25 15:34:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657630>
{'comment': ['0'],
 'name': [' 尤克里里小姐姐'],
 'seen': ['549'],
 'tags': ['少女插画', '清新', '唯美', '海边', '树'],
 'thumb_up': ['10']}
2021-01-25 15:34:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657722> (referer: None)
name [' 上帝让一个人受难，就赐予她美丽']
2021-01-25 15:34:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657722>
{'comment': ['0'],
 'name': [' 上帝让一个人受难，就赐予她美丽'],
 'seen': ['295'],
 'tags': ['厚涂', '少女', '插画'],
 'thumb_up': ['5']}
2021-01-25 15:34:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657632> (referer: None)
name [' 无']
2021-01-25 15:34:37 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657632>
{'comment': ['0'],
 'name': [' 无'],
 'seen': ['185'],
 'tags': ['板绘', '少女'],
 'thumb_up': ['4']}
2021-01-25 15:34:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657745> (referer: None)
2021-01-25 15:34:38 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657745>
{'comment': ['0'],
 'name': [' 原图来自暖暖，衣服很好看被我毁了，可是我不也想的'],
 'seen': ['382'],
 'tags': ['立绘', '临摹'],
 'thumb_up': ['5']}
name [' 原图来自暖暖，衣服很好看被我毁了，可是我不也想的']
2021-01-25 15:34:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657642> (referer: None)
name [' 花花少女']
2021-01-25 15:34:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657642>
{'comment': ['0'],
 'name': [' 花花少女'],
 'seen': ['117'],
 'tags': [],
 'thumb_up': ['4']}
2021-01-25 15:34:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657784> (referer: None)
name [' 学园少女']
2021-01-25 15:34:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657784>
{'comment': ['0'],
 'name': [' 学园少女'],
 'seen': ['197'],
 'tags': ['少女', '学院'],
 'thumb_up': ['5']}
2021-01-25 15:34:42 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657656> (referer: None)
name [' 美人执扇~']
2021-01-25 15:34:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657656>
{'comment': ['0'],
 'name': [' 美人执扇~'],
 'seen': ['419'],
 'tags': ['古典美人图', '汉服少女', '言情小说封面', '汉服美少女', '手绘', '板绘', '古风插画'],
 'thumb_up': ['6']}
2021-01-25 15:34:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657788> (referer: None)
name [' 古风少女']
2021-01-25 15:34:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657788>
{'comment': ['0'],
 'name': [' 古风少女'],
 'seen': ['103'],
 'tags': ['少女'],
 'thumb_up': ['4']}
2021-01-25 15:34:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657657> (referer: None)
2021-01-25 15:34:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657657>
{'comment': ['0'],
 'name': [' 美人儿逗鸟图~'],
 'seen': ['448'],
 'tags': ['古风美人', '板绘', '手绘', '言情插画', '言情女主立绘', '汉服少女', '水彩美人', '小说封面'],
 'thumb_up': ['5']}
name [' 美人儿逗鸟图~']
2021-01-25 15:34:47 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657818> from <GET http://gracg.com/works/view/1657818>
2021-01-25 15:34:48 [scrapy.extensions.logstats] INFO: Crawled 102 pages (at 18 pages/min), scraped 96 items (at 18 items/min)
2021-01-25 15:34:50 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657819> from <GET http://gracg.com/works/view/1657819>
2021-01-25 15:34:51 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657828> from <GET http://gracg.com/works/view/1657828>
2021-01-25 15:34:53 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657839> from <GET http://gracg.com/works/view/1657839>
2021-01-25 15:34:55 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657846> from <GET http://gracg.com/works/view/1657846>
2021-01-25 15:34:57 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657850> from <GET http://gracg.com/works/view/1657850>
2021-01-25 15:34:59 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657942> from <GET http://gracg.com/works/view/1657942>
2021-01-25 15:35:01 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657876> from <GET http://gracg.com/works/view/1657876>
2021-01-25 15:35:03 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657943> from <GET http://gracg.com/works/view/1657943>
2021-01-25 15:35:05 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657899> from <GET http://gracg.com/works/view/1657899>
2021-01-25 15:35:07 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657944> from <GET http://gracg.com/works/view/1657944>
2021-01-25 15:35:09 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657906> from <GET http://gracg.com/works/view/1657906>
2021-01-25 15:35:11 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657946> from <GET http://gracg.com/works/view/1657946>
2021-01-25 15:35:12 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657908> from <GET http://gracg.com/works/view/1657908>
2021-01-25 15:35:15 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657947> from <GET http://gracg.com/works/view/1657947>
2021-01-25 15:35:16 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657912> from <GET http://gracg.com/works/view/1657912>
2021-01-25 15:35:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657818> (referer: None)
name [' 无']
2021-01-25 15:35:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657818>
{'comment': ['0'],
 'name': [' 无'],
 'seen': ['175'],
 'tags': ['板绘', '少女'],
 'thumb_up': ['3']}
2021-01-25 15:35:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657819> (referer: None)
name [' 无1']
2021-01-25 15:35:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657819>
{'comment': ['0'],
 'name': [' 无1'],
 'seen': ['244'],
 'tags': ['板绘', '少女'],
 'thumb_up': ['4']}
2021-01-25 15:35:22 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657828> (referer: None)
2021-01-25 15:35:22 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657828>
{'comment': ['0'],
 'name': [' 【插画】小雪夜'],
 'seen': ['567'],
 'tags': ['原创', '商业插画', '古风', '少女', '插画', '板绘', '场景'],
 'thumb_up': ['4']}
name [' 【插画】小雪夜']
2021-01-25 15:35:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657839> (referer: None)
2021-01-25 15:35:24 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657839>
{'comment': ['0'],
 'name': [' 戴花草帽的少女'],
 'seen': ['310'],
 'tags': ['原创', '唯美', '写实'],
 'thumb_up': ['4']}
name [' 戴花草帽的少女']
2021-01-25 15:35:25 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657846> (referer: None)
2021-01-25 15:35:25 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657846>
{'comment': ['0'],
 'name': [' 阳台'],
 'seen': ['359'],
 'tags': ['打卡', '插画', '原创', '板绘', '少女'],
 'thumb_up': ['8']}
name [' 阳台']
2021-01-25 15:35:26 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657850> (referer: None)
name [' 快乐摸头头']
2021-01-25 15:35:26 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657850>
{'comment': ['0'],
 'name': [' 快乐摸头头'],
 'seen': ['517'],
 'tags': ['打卡', 'procreate', '头像', '少女'],
 'thumb_up': ['7']}
2021-01-25 15:35:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657942> (referer: None)
name [' 少女']
2021-01-25 15:35:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657942>
{'comment': ['0'],
 'name': [' 少女'],
 'seen': ['150'],
 'tags': ['板绘', '商业插画', '唯美', '少女', '插画', '女孩'],
 'thumb_up': ['6']}
2021-01-25 15:35:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657876> (referer: None)
name [' 新画风尝试']
2021-01-25 15:35:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657876>
{'comment': ['0'],
 'name': [' 新画风尝试'],
 'seen': ['356'],
 'tags': ['绘本', '插画', '少女', '儿童插画'],
 'thumb_up': ['4']}
2021-01-25 15:35:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657943> (referer: None)
name [' 少女']
2021-01-25 15:35:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657943>
{'comment': ['0'],
 'name': [' 少女'],
 'seen': ['192'],
 'tags': ['原创', '商业插画', '唯美', '清新', 'procreate'],
 'thumb_up': ['6']}
2021-01-25 15:35:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657899> (referer: None)
2021-01-25 15:35:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657899>
{'comment': ['0'],
 'name': [' 可可爱爱'],
 'seen': ['243'],
 'tags': ['少女', '手绘', '插画'],
 'thumb_up': ['6']}
name [' 可可爱爱']
2021-01-25 15:35:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657944> (referer: None)
name [' 少女']
2021-01-25 15:35:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657944>
{'comment': ['0'],
 'name': [' 少女'],
 'seen': ['152'],
 'tags': ['少女', 'procreate', '唯美', '清新', '板绘', '插画'],
 'thumb_up': ['4']}
2021-01-25 15:35:38 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657906> (referer: None)
name [' 手绘头像']
2021-01-25 15:35:38 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657906>
{'comment': ['0'],
 'name': [' 手绘头像'],
 'seen': ['160'],
 'tags': ['插画', '板绘', '手绘', '少女', '头像'],
 'thumb_up': ['4']}
2021-01-25 15:35:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657946> (referer: None)
name [' 少女']
2021-01-25 15:35:40 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657946>
{'comment': ['0'],
 'name': [' 少女'],
 'seen': ['141'],
 'tags': ['原创', '插画', '少女', '场景', '唯美', '板绘'],
 'thumb_up': ['5']}
2021-01-25 15:35:42 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657908> (referer: None)
2021-01-25 15:35:42 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657908>
{'comment': ['0'],
 'name': [' 少女梦'],
 'seen': ['226'],
 'tags': [],
 'thumb_up': ['4']}
name [' 少女梦']
2021-01-25 15:35:44 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657947> (referer: None)
2021-01-25 15:35:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657947>
{'comment': ['0'],
 'name': [' 少女'],
 'seen': ['133'],
 'tags': ['原创', '插画', '少女', '商业插画', '场景', '清新', '唯美'],
 'thumb_up': ['5']}
name [' 少女']
2021-01-25 15:35:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657912> (referer: None)
2021-01-25 15:35:45 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657912>
{'comment': ['0'],
 'name': [' 少女'],
 'seen': ['261'],
 'tags': ['板绘', '少女', '绘画', '原创', '商业插画', '唯美'],
 'thumb_up': ['4']}
name [' 少女']
2021-01-25 15:35:47 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657951> from <GET http://gracg.com/works/view/1657951>
2021-01-25 15:35:48 [scrapy.extensions.logstats] INFO: Crawled 118 pages (at 16 pages/min), scraped 112 items (at 16 items/min)
2021-01-25 15:35:49 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657914> from <GET http://gracg.com/works/view/1657914>
2021-01-25 15:35:50 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657953> from <GET http://gracg.com/works/view/1657953>
2021-01-25 15:35:52 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657918> from <GET http://gracg.com/works/view/1657918>
2021-01-25 15:35:54 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657959> from <GET http://gracg.com/works/view/1657959>
2021-01-25 15:35:56 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657960> from <GET http://gracg.com/works/view/1657960>
2021-01-25 15:35:58 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657963> from <GET http://gracg.com/works/view/1657963>
2021-01-25 15:36:00 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://gracg.com/works/view/1657965> from <GET http://gracg.com/works/view/1657965>
2021-01-25 15:36:02 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657951> (referer: None)
2021-01-25 15:36:02 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657951>
{'comment': ['0'],
 'name': [' 少女'],
 'seen': ['182'],
 'tags': ['原创', '插画', '少女', '商业插画', '唯美', '清新', '板绘'],
 'thumb_up': ['3']}
name [' 少女']
2021-01-25 15:36:04 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657914> (referer: None)
name [' 少女']
2021-01-25 15:36:04 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657914>
{'comment': ['0'],
 'name': [' 少女'],
 'seen': ['263'],
 'tags': ['原创', '少女', '板绘', '插画', '商业插画', '唯美', '风景'],
 'thumb_up': ['4']}
2021-01-25 15:36:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657953> (referer: None)
name [' 少女合集']
2021-01-25 15:36:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657953>
{'comment': ['0'],
 'name': [' 少女合集'],
 'seen': ['133'],
 'tags': ['插画', '少女', '唯美', '清新', '商业插画', '板绘', 'procreate'],
 'thumb_up': ['3']}
2021-01-25 15:36:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657918> (referer: None)
2021-01-25 15:36:07 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657918>
{'comment': ['0'],
 'name': [' 少女'],
 'seen': ['219'],
 'tags': ['板绘', '少女', '商业插画', '唯美', '清新', 'Procreate'],
 'thumb_up': ['4']}
name [' 少女']
2021-01-25 15:36:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657959> (referer: None)
name [' 粉色少女心']
2021-01-25 15:36:08 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657959>
{'comment': ['0'],
 'name': [' 粉色少女心'],
 'seen': ['411'],
 'tags': ['原创', '插画', '少女', '板绘', '女孩', '可爱'],
 'thumb_up': ['12']}
2021-01-25 15:36:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657960> (referer: None)
2021-01-25 15:36:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657960>
{'comment': ['0'],
 'name': [' 1yue'],
 'seen': ['1222'],
 'tags': ['商业插画', '唯美', '插画', '原创'],
 'thumb_up': ['22']}
name [' 1yue']
2021-01-25 15:36:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657963> (referer: None)
name [' 坠落之星']
2021-01-25 15:36:12 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657963>
{'comment': ['0'],
 'name': [' 坠落之星'],
 'seen': ['191'],
 'tags': ['插图', '原创', '绘本', 'procreate', '板绘', '少女'],
 'thumb_up': ['5']}
2021-01-25 15:36:14 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://gracg.com/works/view/1657965> (referer: None)
2021-01-25 15:36:14 [scrapy.core.scraper] DEBUG: Scraped from <200 https://gracg.com/works/view/1657965>
{'comment': ['0'],
 'name': [' 新年贺图/牛年大吉'],
 'seen': ['216'],
 'tags': ['插画', '贺图', '新年', '牛年大吉', '板绘', '少女', '可爱', '甜美'],
 'thumb_up': ['3']}
2021-01-25 15:36:14 [scrapy.core.engine] INFO: Closing spider (finished)
2021-01-25 15:36:14 [selenium.webdriver.remote.remote_connection] DEBUG: DELETE http://127.0.0.1:4240/session/c2c7230067140857e92286f77fa8b307 {}
2021-01-25 15:36:14 [urllib3.connectionpool] DEBUG: http://127.0.0.1:4240 "DELETE /session/c2c7230067140857e92286f77fa8b307 HTTP/1.1" 200 14
name [' 新年贺图/牛年大吉']
2021-01-25 15:36:14 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2021-01-25 15:36:16 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 57863,
 'downloader/request_count': 246,
 'downloader/request_method_count/GET': 246,
 'downloader/response_bytes': 2742943,
 'downloader/response_count': 246,
 'downloader/response_status_count/200': 126,
 'downloader/response_status_count/301': 120,
 'elapsed_time_seconds': 446.315551,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 1, 25, 7, 36, 14, 495976),
 'item_scraped_count': 120,
 'log_count/DEBUG': 888,
 'log_count/INFO': 17,
 'request_depth_max': 1,
 'response_received_count': 126,
 'robotstxt/request_count': 1,
 'robotstxt/response_count': 1,
 'robotstxt/response_status_count/200': 1,
 'scheduler/dequeued': 245,
 'scheduler/dequeued/memory': 245,
 'scheduler/enqueued': 245,
 'scheduler/enqueued/memory': 245,
 'start_time': datetime.datetime(2021, 1, 25, 7, 28, 48, 180425)}
2021-01-25 15:36:16 [scrapy.core.engine] INFO: Spider closed (finished)


{"name": ["唱会啦"], "seen": ["34"], "thumb_up": ["0"], "comment": ["0"], "tags": ["板绘", "手绘", "少女", "人物", "Q版"]}
{"name": [" 古装少女插画"], "seen": ["144"], "thumb_up": ["0"], "comment": ["0"], "tags": ["古风", "少女", "发饰", "竹叶"]}
{"name": [" 【人物头像】那个女人"], "seen": ["114"], "thumb_up": ["2"], "comment": ["0"], "tags": ["插画", "头像", "少女心", "每日一画", "打卡", "儿童插画"]}
{"name": [" 临摹"], "seen": ["154"], "thumb_up": ["3"], "comment": ["0"], "tags": ["procreate", "打卡", "少女", "人物", "练习", "临摹"]}
{"name": [" 原来你是这样的葡萄"], "seen": ["170"], "thumb_up": ["3"], "comment": ["0"], "tags": ["插画", "少女", "练习"]}
{"name": [" 漫画版簪花仕女"], "seen": ["67"], "thumb_up": ["3"], "comment": ["0"], "tags": ["绘本", "古风", "少女", "插画", "可爱", "Q版", "中国风"]}
{"name": [" 路过冬季街头咖啡小铺"], "seen": ["213"], "thumb_up": ["5"], "comment": ["0"], "tags": ["板绘", "冬天", "原创", "街景", "少女插画", "打卡", "水彩", "绘本", "手绘", "ps", "每日一画"]}
{"name": [" 少女插画"], "seen": ["198"], "thumb_up": ["4"], "comment": ["0"], "tags": ["插画", "清新"]}
{"name": [" 古风人物厚涂插画"], "seen": ["142"], "thumb_up": ["5"], "comment": ["0"], "tags": ["板绘", "古风", "少女", "厚涂"]}
{"name": [" procreate插画｜少女下午茶"], "seen": ["147"], "thumb_up": ["5"], "comment": ["0"], "tags": ["清新", "绿植", "少女", "procreate", "插画"]}
{"name": [" 幻书启世录"], "seen": ["172"], "thumb_up": ["3"], "comment": ["0"], "tags": ["幻书启世录", "少女"]}
{"name": [" 十二生肖拟人系列"], "seen": ["183"], "thumb_up": ["4"], "comment": ["0"], "tags": ["国风", "人物", "插画", "传统", "生肖"]}
{"name": [" 叮～～"], "seen": ["132"], "thumb_up": ["4"], "comment": ["0"], "tags": ["每日一画", "打卡", "少女", "可爱", "Q版", "人设", "二次元"]}
{"name": [" 关于贵族少女是否流泪的这件事"], "seen": ["268"], "thumb_up": ["3"], "comment": ["0"], "tags": []}
{"name": [" 大明风华"], "seen": ["442"], "thumb_up": ["6"], "comment": ["1"], "tags": ["影视", "人物", "壁纸", "绘画", "插画", "oc", "水彩"]}
{"name": [" Biubiu~"], "seen": ["243"], "thumb_up": ["5"], "comment": ["0"], "tags": ["头像", "少女", "厚涂"]}
{"name": [" 【人物头像】少女"], "seen": ["305"], "thumb_up": ["4"], "comment": ["0"], "tags": ["原创", "打卡", "每日一画", "少女", "插画", "板绘", "头像"]}
{"name": [" 桃花妖"], "seen": ["336"], "thumb_up": ["3"], "comment": ["0"], "tags": ["插画", "少女"]}
{"name": [" 少女穿搭"], "seen": ["403"], "thumb_up": ["3"], "comment": ["0"], "tags": ["少女", "可爱", "人物", "清新"]}
{"name": [" 版权保护"], "seen": ["280"], "thumb_up": ["4"], "comment": ["0"], "tags": ["少女", "插画", "女孩", "涂鸦"]}
{"name": [" 坚持摸鱼（1/100）"], "seen": ["587"], "thumb_up": ["10"], "comment": ["0"], "tags": ["打卡", "procreate", "插画", "原创", "少女", "板绘", "头像"]}
{"name": [" 尝试学习国画创作ing……"], "seen": ["368"], "thumb_up": ["5"], "comment": ["0"], "tags": ["procreate", "水彩", "原创", "少女", "水墨"]}
{"name": [" 无题"], "seen": ["321"], "thumb_up": ["5"], "comment": ["0"], "tags": ["美少女", "绘画"]}
{"name": [" 【原创】向日葵"], "seen": ["183"], "thumb_up": ["4"], "comment": ["0"], "tags": ["绘本", "打卡", "少女", "可爱", "儿童插画", "Q版"]}
{"name": [" 祝你新年快乐呀"], "seen": ["477"], "thumb_up": ["5"], "comment": ["0"], "tags": ["procreate", "板绘", "少女", "头像", "手绘", "绘画", "人物"]}
{"name": [" 精致少女"], "seen": ["1074"], "thumb_up": ["15"], "comment": ["0"], "tags": []}
{"name": [" 我们都在海里"], "seen": ["462"], "thumb_up": ["5"], "comment": ["0"], "tags": ["插画", "原创", "板绘", "少女", "绘画", "日系", "二次元"]}
{"name": [" 森女系插画"], "seen": ["320"], "thumb_up": ["4"], "comment": ["0"], "tags": ["手绘", "少女"]}
{"name": [" 星空下的少女"], "seen": ["400"], "thumb_up": ["5"], "comment": ["0"], "tags": []}
{"name": [" ►"], "seen": ["320"], "thumb_up": ["4"], "comment": ["0"], "tags": ["少女", "Procreate", "打卡", "女孩", "人物", "绘画"]}
{"name": [" 一个单眼皮朋友"], "seen": ["334"], "thumb_up": ["4"], "comment": ["0"], "tags": ["闺蜜头像", "少女插画", "插画"]}
{"name": [" 旋"], "seen": ["4184"], "thumb_up": ["103"], "comment": ["10"], "tags": ["版绘", "少女", "双人", "小清新", "游乐园", "旋转木马"]}
{"name": [" 不良少女"], "seen": ["232"], "thumb_up": ["5"], "comment": ["0"], "tags": []}
{"name": [" 《等风》"], "seen": ["1964"], "thumb_up": ["40"], "comment": ["0"], "tags": ["原创", "绘画", "少女", "场景", "唯美", "清新", "治愈"]}
{"name": [" 随便画画"], "seen": ["222"], "thumb_up": ["4"], "comment": ["0"], "tags": ["头像", "少女", "女孩", "可爱", "厚涂"]}
{"name": [" 樱桃"], "seen": ["1575"], "thumb_up": ["33"], "comment": ["4"], "tags": ["少女", "樱桃", "粉色", "每日一画", "礼物"]}
{"name": [" 情侣色块插画"], "seen": ["1106"], "thumb_up": ["15"], "comment": ["0"], "tags": ["青春", "少女", "场景"]}
{"name": [" 棒棒糖少女"], "seen": ["314"], "thumb_up": ["5"], "comment": ["0"], "tags": []}
{"name": [" 戏水"], "seen": ["289"], "thumb_up": ["4"], "comment": ["0"], "tags": ["少女", "治愈", "插画"]}
{"name": [" 自创人物头像"], "seen": ["264"], "thumb_up": ["4"], "comment": ["0"], "tags": ["少女", "绘画", "人物", "Procreate", "女孩"]}
{"name": [" 数字创作3"], "seen": ["423"], "thumb_up": ["5"], "comment": ["0"], "tags": ["原创", "procreate", "商业插画", "场景", "唯美", "人像", "少女"]}
{"name": [" 深海蓝鱼"], "seen": ["143"], "thumb_up": ["5"], "comment": ["0"], "tags": ["插画", "手绘", "封面", "原创", "Procreate"]}
{"name": [" 愉悦的步伐"], "seen": ["228"], "thumb_up": ["3"], "comment": ["0"], "tags": ["procreate", "插画", "原创", "少女", "涂鸦"]}
{"name": [" 林中少女"], "seen": ["186"], "thumb_up": ["4"], "comment": ["0"], "tags": ["原创", "插画", "板绘", "绘画", "少女", "植物"]}
{"name": [" 鸽子"], "seen": ["180"], "thumb_up": ["3"], "comment": ["0"], "tags": ["二次元", "插画", "板绘", "求约稿", "厚涂", "美少女"]}
{"name": [" 海蝴蝶"], "seen": ["200"], "thumb_up": ["4"], "comment": ["0"], "tags": ["人鱼", "封面", "手绘", "procreate"]}
{"name": [" 静静看着你"], "seen": ["435"], "thumb_up": ["4"], "comment": ["0"], "tags": ["可爱", "女孩", "少女", "板绘"]}
{"name": [" 白日梦"], "seen": ["255"], "thumb_up": ["5"], "comment": ["0"], "tags": ["封面", "白日梦", "procreate", "手绘", "绘图", "插画"]}
{"name": [" 手绘头像"], "seen": ["273"], "thumb_up": ["4"], "comment": ["0"], "tags": ["原创", "少女", "古风", "头像", "可爱"]}
{"name": [" Twice 林娜琏&amp;SANA"], "seen": ["237"], "thumb_up": ["4"], "comment": ["0"], "tags": ["偶像", "少女"]}
{"name": [" 心有猛虎，细嗅蔷薇"], "seen": ["496"], "thumb_up": ["5"], "comment": ["0"], "tags": ["手绘", "少女", "原创", "人物", "女孩", "清新", "植物"]}
{"name": [" 水彩"], "seen": ["416"], "thumb_up": ["5"], "comment": ["0"], "tags": ["水彩", "女孩", "可爱", "商业插画", "绘本", "设计", "治愈"]}
{"name": [" 无题"], "seen": ["237"], "thumb_up": ["4"], "comment": ["0"], "tags": ["少女", "手绘"]}
{"name": [" 《十二生肖-羊》"], "seen": ["455"], "thumb_up": ["5"], "comment": ["0"], "tags": ["打卡", "插画", "原创", "板绘", "绘画", "少女", "人物"]}
{"name": [" 茉莉花茶"], "seen": ["842"], "thumb_up": ["6"], "comment": ["0"], "tags": ["少女", "水彩", "插画"]}
{"name": [" 今天就画一张人像手机壁纸吧"], "seen": ["284"], "thumb_up": ["6"], "comment": ["0"], "tags": ["每日一画", "原创", "少女", "人像", "二次元", "女孩", "帅气"]}
{"name": [" 女孩子"], "seen": ["1567"], "thumb_up": ["23"], "comment": ["0"], "tags": ["插画"]}
{"name": [" "], "seen": ["194"], "thumb_up": ["4"], "comment": ["0"], "tags": ["procreate", "原创", "插画", "少女", "扁平风"]}
{"name": [" 画照片"], "seen": ["344"], "thumb_up": ["6"], "comment": ["0"], "tags": ["手绘", "绘画", "头像", "少女"]}
{"name": [" 没有收尾"], "seen": ["302"], "thumb_up": ["4"], "comment": ["0"], "tags": ["头像", "少女", "女孩", "人像", "ipad", "可爱"]}
{"name": [" 日常"], "seen": ["612"], "thumb_up": ["15"], "comment": ["0"], "tags": ["少女"]}
{"name": [" 猫猫女"], "seen": ["247"], "thumb_up": ["4"], "comment": ["0"], "tags": ["原创", "插画", "少女", "人物", "sai", "日系"]}
{"name": [" 临摹"], "seen": ["289"], "thumb_up": ["5"], "comment": ["0"], "tags": []}
{"name": [" 抹茶温泉"], "seen": ["343"], "thumb_up": ["4"], "comment": ["0"], "tags": ["手绘", "水彩", "少女", "梦幻", "可爱"]}
{"name": [" 临摹"], "seen": ["274"], "thumb_up": ["4"], "comment": ["0"], "tags": []}
{"name": [" 新生"], "seen": ["303"], "thumb_up": ["4"], "comment": ["0"], "tags": ["插画", "原创", "少女", "人物", "女孩"]}
{"name": [" 白天的月亮"], "seen": ["340"], "thumb_up": ["5"], "comment": ["0"], "tags": ["插画", "原创", "人物", "每日一画", "打卡", "少女", "绘画"]}
{"name": [" 少女"], "seen": ["289"], "thumb_up": ["5"], "comment": ["0"], "tags": []}
{"name": [" 【人物头像】少女"], "seen": ["345"], "thumb_up": ["5"], "comment": ["0"], "tags": ["原创", "插画", "儿童插画", "少女", "每日一画", "头像", "板绘"]}
{"name": [" 水彩练习"], "seen": ["235"], "thumb_up": ["5"], "comment": ["0"], "tags": ["水彩", "临摹", "少女"]}
{"name": [" 水彩，女孩与蛇"], "seen": ["276"], "thumb_up": ["4"], "comment": ["0"], "tags": ["水彩", "少女", "蛇", "手绘"]}
{"name": [" 落书"], "seen": ["303"], "thumb_up": ["5"], "comment": ["0"], "tags": ["少女", "插画", "板绘"]}
{"name": [" 向日葵和少女"], "seen": ["687"], "thumb_up": ["11"], "comment": ["0"], "tags": ["清新", "扁平", "植物", "板绘"]}
{"name": [" 水彩人物"], "seen": ["349"], "thumb_up": ["4"], "comment": ["0"], "tags": ["水彩", "绘本", "少女", "插画", "清新", "治愈", "植物"]}
{"name": [" 眼镜娘"], "seen": ["198"], "thumb_up": ["5"], "comment": ["0"], "tags": ["插画", "美少女"]}
{"name": [" "], "seen": ["190"], "thumb_up": ["5"], "comment": ["0"], "tags": ["procreate", "原创", "插画", "头像", "少女", "绘画"]}
{"name": [" 数字创作4"], "seen": ["413"], "thumb_up": ["7"], "comment": ["0"], "tags": ["原创", "插画", "少女", "商业插画", "场景"]}
{"name": [" 水彩插画"], "seen": ["323"], "thumb_up": ["5"], "comment": ["0"], "tags": ["每日一画", "少女", "清新", "治愈"]}
{"name": [" 光影女孩"], "seen": ["360"], "thumb_up": ["10"], "comment": ["0"], "tags": ["每日一画", "板绘", "少女", "头像"]}
{"name": [" 周末少女"], "seen": ["187"], "thumb_up": ["5"], "comment": ["0"], "tags": ["原创", "少女", "绘画", "女孩", "涂鸦", "绘本", "漫画"]}
{"name": [" 小圆"], "seen": ["108"], "thumb_up": ["4"], "comment": ["0"], "tags": ["小圆"]}
{"name": [" 黑历史重绘"], "seen": ["2332"], "thumb_up": ["33"], "comment": ["2"], "tags": ["procreate", "古风", "头像", "少女", "人物", "中国风", "女孩"]}
{"name": [" 人设-花"], "seen": ["391"], "thumb_up": ["6"], "comment": ["0"], "tags": ["原创", "古风", "少女"]}
{"name": [" 【2020下半年印花图案设计作品合集】"], "seen": ["306"], "thumb_up": ["4"], "comment": ["0"], "tags": ["印花", "服装印花", "碎花", "可爱", "少女"]}
{"name": [" 一些图！"], "seen": ["103"], "thumb_up": ["5"], "comment": ["0"], "tags": ["魔法少女小圆", "物语系列", "忍野忍", "午睡留声机", "明日之子", "头像", "约稿", "摸鱼", "巴麻美", "鹿目圆香"]}
{"name": [" 一个可爱小姐姐的约稿"], "seen": ["258"], "thumb_up": ["5"], "comment": ["0"], "tags": ["插画", "原创", "少女", "头像", "女孩", "可爱"]}
{"name": [" 梦幻小女巫~"], "seen": ["256"], "thumb_up": ["6"], "comment": ["0"], "tags": ["原创", "少女", "可爱", "星空"]}
{"name": [" 尤克里里小姐姐"], "seen": ["549"], "thumb_up": ["10"], "comment": ["0"], "tags": ["少女插画", "清新", "唯美", "海边", "树"]}
{"name": [" 上帝让一个人受难，就赐予她美丽"], "seen": ["295"], "thumb_up": ["5"], "comment": ["0"], "tags": ["厚涂", "少女", "插画"]}
{"name": [" 无"], "seen": ["185"], "thumb_up": ["4"], "comment": ["0"], "tags": ["板绘", "少女"]}
{"name": [" 原图来自暖暖，衣服很好看被我毁了，可是我不也想的"], "seen": ["382"], "thumb_up": ["5"], "comment": ["0"], "tags": ["立绘", "临摹"]}
{"name": [" 花花少女"], "seen": ["117"], "thumb_up": ["4"], "comment": ["0"], "tags": []}
{"name": [" 学园少女"], "seen": ["197"], "thumb_up": ["5"], "comment": ["0"], "tags": ["少女", "学院"]}
{"name": [" 美人执扇~"], "seen": ["419"], "thumb_up": ["6"], "comment": ["0"], "tags": ["古典美人图", "汉服少女", "言情小说封面", "汉服美少女", "手绘", "板绘", "古风插画"]}
{"name": [" 古风少女"], "seen": ["103"], "thumb_up": ["4"], "comment": ["0"], "tags": ["少女"]}
{"name": [" 美人儿逗鸟图~"], "seen": ["448"], "thumb_up": ["5"], "comment": ["0"], "tags": ["古风美人", "板绘", "手绘", "言情插画", "言情女主立绘", "汉服少女", "水彩美人", "小说封面"]}
{"name": [" 无"], "seen": ["175"], "thumb_up": ["3"], "comment": ["0"], "tags": ["板绘", "少女"]}
{"name": [" 无1"], "seen": ["244"], "thumb_up": ["4"], "comment": ["0"], "tags": ["板绘", "少女"]}
{"name": [" 【插画】小雪夜"], "seen": ["567"], "thumb_up": ["4"], "comment": ["0"], "tags": ["原创", "商业插画", "古风", "少女", "插画", "板绘", "场景"]}
{"name": [" 戴花草帽的少女"], "seen": ["310"], "thumb_up": ["4"], "comment": ["0"], "tags": ["原创", "唯美", "写实"]}
{"name": [" 阳台"], "seen": ["359"], "thumb_up": ["8"], "comment": ["0"], "tags": ["打卡", "插画", "原创", "板绘", "少女"]}
{"name": [" 快乐摸头头"], "seen": ["517"], "thumb_up": ["7"], "comment": ["0"], "tags": ["打卡", "procreate", "头像", "少女"]}
{"name": [" 少女"], "seen": ["150"], "thumb_up": ["6"], "comment": ["0"], "tags": ["板绘", "商业插画", "唯美", "少女", "插画", "女孩"]}
{"name": [" 新画风尝试"], "seen": ["356"], "thumb_up": ["4"], "comment": ["0"], "tags": ["绘本", "插画", "少女", "儿童插画"]}
{"name": [" 少女"], "seen": ["192"], "thumb_up": ["6"], "comment": ["0"], "tags": ["原创", "商业插画", "唯美", "清新", "procreate"]}
{"name": [" 可可爱爱"], "seen": ["243"], "thumb_up": ["6"], "comment": ["0"], "tags": ["少女", "手绘", "插画"]}
{"name": [" 少女"], "seen": ["152"], "thumb_up": ["4"], "comment": ["0"], "tags": ["少女", "procreate", "唯美", "清新", "板绘", "插画"]}
{"name": [" 手绘头像"], "seen": ["160"], "thumb_up": ["4"], "comment": ["0"], "tags": ["插画", "板绘", "手绘", "少女", "头像"]}
{"name": [" 少女"], "seen": ["141"], "thumb_up": ["5"], "comment": ["0"], "tags": ["原创", "插画", "少女", "场景", "唯美", "板绘"]}
{"name": [" 少女梦"], "seen": ["226"], "thumb_up": ["4"], "comment": ["0"], "tags": []}
{"name": [" 少女"], "seen": ["133"], "thumb_up": ["5"], "comment": ["0"], "tags": ["原创", "插画", "少女", "商业插画", "场景", "清新", "唯美"]}
{"name": [" 少女"], "seen": ["261"], "thumb_up": ["4"], "comment": ["0"], "tags": ["板绘", "少女", "绘画", "原创", "商业插画", "唯美"]}
{"name": [" 少女"], "seen": ["182"], "thumb_up": ["3"], "comment": ["0"], "tags": ["原创", "插画", "少女", "商业插画", "唯美", "清新", "板绘"]}
{"name": [" 少女"], "seen": ["263"], "thumb_up": ["4"], "comment": ["0"], "tags": ["原创", "少女", "板绘", "插画", "商业插画", "唯美", "风景"]}
{"name": [" 少女合集"], "seen": ["133"], "thumb_up": ["3"], "comment": ["0"], "tags": ["插画", "少女", "唯美", "清新", "商业插画", "板绘", "procreate"]}
{"name": [" 少女"], "seen": ["219"], "thumb_up": ["4"], "comment": ["0"], "tags": ["板绘", "少女", "商业插画", "唯美", "清新", "Procreate"]}
{"name": [" 粉色少女心"], "seen": ["411"], "thumb_up": ["12"], "comment": ["0"], "tags": ["原创", "插画", "少女", "板绘", "女孩", "可爱"]}
{"name": [" 1yue"], "seen": ["1222"], "thumb_up": ["22"], "comment": ["0"], "tags": ["商业插画", "唯美", "插画", "原创"]}
{"name": [" 坠落之星"], "seen": ["191"], "thumb_up": ["5"], "comment": ["0"], "tags": ["插图", "原创", "绘本", "procreate", "板绘", "少女"]}
{"name": [" 新年贺图/牛年大吉"], "seen": ["216"], "thumb_up": ["3"], "comment": ["0"], "tags": ["插画", "贺图", "新年", "牛年大吉", "板绘", "少女", "可爱", "甜美"]}


#利用pandas和matplotlib将爬取下来的数据做一个简单的可视化
import pandas
import matplotlib.pyplot as plt
from pylab import mpl

#mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['font.sans-serif'] = ['Microsoft YaHei']    # 指定默认字体：解决plot不能显示中文问题
mpl.rcParams['axes.unicode_minus'] = False           # 解决保存图像是负号'-'显示为方块的问题

#读取pandas文件
df = pandas.read_json(path_or_buf="data.json", orient='records',lines=True, encoding='utf-8',convert_dates=False)
print(type(df['tags'][0]))
print(type(df))

#利用字典来统计tag出现的数量
count_dict = dict()
for index,row in df.iterrows():
    for item in row[-1]:
        if item in count_dict:
            count_dict[item] += 1
        else:
            count_dict[item] = 1
#print(count_dict)
ser = pandas.Series(count_dict)
 print(ser)

#排序
ser.sort_values(ascending=False, inplace=True)
#画图
plt.figure()
ser.plot(kind='bar')
plt.title('标签出现数量统计')
plt.show()
